Explore, segment, and cluster the neighborhoods in the city of Toronto

First import the libraries we need

In [74]:
#Un-comment if needed
#!pip install beautifulsoup4
#!pip install lxml
#!pip install requests

#Import libraries

from bs4 import BeautifulSoup
import requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#import json # library to handle JSON files

#import csv # library to read/write csv files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Now download the data for Toronto neighborhoods

In [75]:
!wget -q -O 'toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


Use BeautifulSoup to extract data from downloaded data

In [76]:
with open('toronto_data.html') as html_file:
    tsoup = BeautifulSoup(html_file,'lxml')
#print(toronto_soup.prettify())

In [77]:
datacols = []
for th in tsoup.find_all('th'):
    datacols.append(th.text.rstrip())
    


In [78]:
alldata = []
datarows = []
for td in tsoup.find_all('td'):
    alldata.append(td.text.rstrip())

rowlen = len(datacols)-1
numrows = int(len(alldata)/rowlen)

for rnum in range(0,numrows):
    row=[]
    for i in range(0,rowlen):
        row.append(alldata[i+rnum*rowlen])
    if (len(row[0])==3):
        datarows.append(row)



Now create Dataframe from extracted data and remove rows with Borough = Not assigned

In [79]:
df = pd.DataFrame(columns=datacols[0:3])
df2 = pd.DataFrame(datarows,columns=datacols[0:3])
dfToronto=df.append(df2)

dfToronto['Borough'] = dfToronto['Borough'].replace('Not assigned',np.nan)

dfToronto.dropna(axis=0, inplace=True)

dfToronto.head(10)



,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Now find which rows have multiple neighborhoods for same postcode

In [80]:
dup=dfToronto.duplicated(subset='Postcode')
ddup = dup.to_dict()
dupt = []
for key in ddup:
    if ddup[key]:
        dupt.append(key)
    

Now get the list of postcodes which have more than one neighborhood and remove duplicates.

In [81]:
dbor = []
for index in dupt:
    dbor.append(dfToronto.loc[(index,'Postcode')])
dbor1 = list(set(dbor))
print(dbor1)


['M4L', 'M1N', 'M6S', 'M8Z', 'M5R', 'M3C', 'M5T', 'M1L', 'M3K', 'M1R', 'M4V', 'M9V', 'M4X', 'M1K', 'M5H', 'M6L', 'M8X', 'M1B', 'M6J', 'M5V', 'M4T', 'M9M', 'M5X', 'M1P', 'M8Y', 'M1W', 'M5B', 'M9B', 'M5M', 'M1T', 'M1V', 'M5S', 'M6N', 'M8W', 'M1M', 'M6H', 'M5L', 'M5A', 'M6M', 'M4B', 'M4K', 'M5K', 'M2L', 'M6A', 'M8V', 'M1C', 'M5P', 'M5J', 'M6R', 'M6P', 'M3H', 'M2M', 'M3J', 'M2J', 'M6K', 'M1E', 'M9R', 'M9C']



Create dictionary of dataframe rows which have multiple neighborhoods

In [82]:
mn = {}
for pc in dbor1:
    nlist = []
    for index,row in dfToronto.iterrows():
        if row["Postcode"] == pc:
            nlist.append(row["Neighbourhood"])
    mn[pc] = nlist
    
        

Drop duplicates from dataframe, based on postcode

In [83]:
dfT1 = dfToronto
dfT1.drop_duplicates(subset='Postcode',inplace=True)
dfT1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
8,M7A,Queen's Park,Not assigned


Now replace the neighborhoods with lists from dictionary and assign Borough to Not Assigned neighborhoods

In [84]:
for pc in dbor1:
    nlist = mn[pc]
    nstr = ','.join(nlist)
    for index,row in dfT1.iterrows():
        if row["Postcode"] == pc:
            row["Neighbourhood"] = nstr
        if row["Neighbourhood"] == "Not assigned":
            row["Neighbourhood"] = row["Borough"]

dfT1.head(10)       


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


Reset the dataframe index 

In [85]:
dfT1.reset_index(drop=True,inplace=True)
dfT1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Just to be picky change Neighbourhood to conform with assignment instructions  

In [86]:
dfT1.rename(columns={'Neighbourhood': 'Neighborhood'},inplace=True)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [88]:
dfT1

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [87]:
dfT1.shape

(103, 3)